In [16]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from sklearn.metrics import mean_absolute_error, mean_squared_error


In [18]:
X_train = np.load("/content/X_train.npy")
y_train = np.load("/content/y_train.npy")

X_val = np.load("/content/X_val.npy")
y_val = np.load("/content/y_val.npy")

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)


(14161, 30, 17) (14161,)
(3470, 30, 17) (3470,)


In [19]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)

X_val_t = torch.tensor(X_val, dtype=torch.float32)
y_val_t = torch.tensor(y_val, dtype=torch.float32)


In [20]:
BATCH_SIZE = 64

train_loader = DataLoader(
    TensorDataset(X_train_t, y_train_t),
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_loader = DataLoader(
    TensorDataset(X_val_t, y_val_t),
    batch_size=BATCH_SIZE,
    shuffle=False
)


In [21]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        out = self.fc(h_n[-1])
        return out.squeeze()


In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = LSTMModel(input_size=X_train.shape[2]).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [23]:
EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_preds, val_true = [], []

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            preds = model(X_batch).cpu().numpy()
            val_preds.extend(preds)
            val_true.extend(y_batch.numpy())

    val_mae = mean_absolute_error(val_true, val_preds)

    print(f"Epoch {epoch+1}/{EPOCHS} | Train MSE: {train_loss:.4f} | Val MAE: {val_mae:.2f}")


Epoch 1/20 | Train MSE: 11022.8153 | Val MAE: 74.90
Epoch 2/20 | Train MSE: 8680.6321 | Val MAE: 63.18
Epoch 3/20 | Train MSE: 7026.8276 | Val MAE: 54.20
Epoch 4/20 | Train MSE: 5683.4114 | Val MAE: 47.26
Epoch 5/20 | Train MSE: 4646.8134 | Val MAE: 40.53
Epoch 6/20 | Train MSE: 3835.1603 | Val MAE: 35.31
Epoch 7/20 | Train MSE: 3186.9123 | Val MAE: 31.13
Epoch 8/20 | Train MSE: 2707.9778 | Val MAE: 28.18
Epoch 9/20 | Train MSE: 2290.9771 | Val MAE: 25.51
Epoch 10/20 | Train MSE: 2003.6171 | Val MAE: 23.83
Epoch 11/20 | Train MSE: 1743.7381 | Val MAE: 21.53
Epoch 12/20 | Train MSE: 1551.3332 | Val MAE: 20.91
Epoch 13/20 | Train MSE: 1385.3790 | Val MAE: 20.93
Epoch 14/20 | Train MSE: 1256.8474 | Val MAE: 20.70
Epoch 15/20 | Train MSE: 1127.8488 | Val MAE: 19.07
Epoch 16/20 | Train MSE: 1035.8217 | Val MAE: 19.22
Epoch 17/20 | Train MSE: 958.5965 | Val MAE: 19.69
Epoch 18/20 | Train MSE: 901.3796 | Val MAE: 19.28
Epoch 19/20 | Train MSE: 834.1921 | Val MAE: 18.92
Epoch 20/20 | Train MSE

In [25]:
rmse = np.sqrt(mean_squared_error(val_true, val_preds))
print("Validation RMSE:", rmse)


Validation RMSE: 29.746715728974927


In [26]:
torch.save(model.state_dict(), "/content/lstm_model.pth")
